In [127]:
from sklearn.datasets import load_boston
import pandas as pd
data = pd.read_csv('/Users/max/Downloads/NETOLOGY/DVP/дз/data/aac_shelter_outcomes.csv')
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
import datetime
from datetime import datetime
from datetime import timedelta

In [ ]:
# 1) Сразу можно сказать, что удалить смело можно столбец animal_id это просто номер.
# 2) Создаю новый признак возраст животного в днях на основании дат. Удаляем лишние столбцы 'date_of_birth','datetime','monthyear','age_upon_outcome'
# 3) 

In [128]:
data.head()

,age_upon_outcome,animal_id,animal_type,breed,color,date_of_birth,datetime,monthyear,name,outcome_subtype,outcome_type,sex_upon_outcome
0,2 weeks,A684346,Cat,Domestic Shorthair Mix,Orange Tabby,2014-07-07T00:00:00,2014-07-22T16:04:00,2014-07-22T16:04:00,NaN,Partner,Transfer,Intact Male
1,1 year,A666430,Dog,Beagle Mix,White/Brown,2012-11-06T00:00:00,2013-11-07T11:47:00,2013-11-07T11:47:00,Lucy,Partner,Transfer,Spayed Female
2,1 year,A675708,Dog,Pit Bull,Blue/White,2013-03-31T00:00:00,2014-06-03T14:20:00,2014-06-03T14:20:00,*Johnny,NaN,Adoption,Neutered Male
3,9 years,A680386,Dog,Miniature Schnauzer Mix,White,2005-06-02T00:00:00,2014-06-15T15:50:00,2014-06-15T15:50:00,Monday,Partner,Transfer,Neutered Male
4,5 months,A683115,Other,Bat Mix,Brown,2014-01-07T00:00:00,2014-07-07T14:04:00,2014-07-07T14:04:00,NaN,Rabies Risk,Euthanasia,Unknown


In [129]:
data = data[(data['outcome_type'] == 'Transfer') | (data['outcome_type'] ==  'Adoption')]
data.shape

(56611, 12)

In [130]:
data['outcome_type'].replace(['Transfer','Adoption'],[0,1], inplace=True)

In [ ]:
data = data.drop('animal_id', axis=1)

In [189]:
def animal_age_days(data):
    birth, date = data['date_of_birth'], data['datetime']
    age_days = datetime.strptime(date, '%Y-%m-%dT%H:%M:%S') - datetime.strptime(birth, '%Y-%m-%dT%H:%M:%S')
    if age_days.days == 0:
        return 1
    else:
        return age_days.days

In [190]:
data['animal_age_in_days'] = data.apply(animal_age_days, axis=1)

KeyError: ('date_of_birth', 'occurred at index 0')

In [184]:
data = data.drop(['date_of_birth','datetime','monthyear'], axis=1)

In [187]:
data = data.drop('age_upon_outcome', axis=1)

In [191]:
data.head()  

,animal_type,breed,color,name,outcome_subtype,outcome_type,sex_upon_outcome,animal_age_in_days
0,Cat,Domestic Shorthair Mix,Orange Tabby,NaN,Partner,0,Intact Male,15
1,Dog,Beagle Mix,White/Brown,Lucy,Partner,0,Spayed Female,366
2,Dog,Pit Bull,Blue/White,*Johnny,NaN,1,Neutered Male,429
3,Dog,Miniature Schnauzer Mix,White,Monday,Partner,0,Neutered Male,3300
5,Dog,Leonberger Mix,Brown/White,*Edgar,Partner,0,Intact Male,126


In [180]:
min(data['animal_age_in_days'])

-123

In [181]:
data.sort_values(by='animal_age_in_days').head(10)

,age_upon_outcome,animal_type,breed,color,date_of_birth,datetime,monthyear,name,outcome_subtype,outcome_type,sex_upon_outcome,animal_age_in_days
19814,0 years,Dog,Labrador Retriever Mix,Black/White,2014-04-03T00:00:00,2013-12-01T13:19:00,2013-12-01T13:19:00,Sadie,Partner,0,Spayed Female,-123
64745,0 years,Cat,Domestic Shorthair Mix,Black,2015-08-29T00:00:00,2015-05-24T17:01:00,2015-05-24T17:01:00,Penelope,NaN,1,Spayed Female,-97
22804,0 years,Dog,Border Collie Mix,Black/White,2014-12-12T00:00:00,2014-09-10T17:29:00,2014-09-10T17:29:00,Juan,NaN,1,Neutered Male,-93
52977,0 years,Cat,Domestic Shorthair Mix,Orange Tabby,2016-10-28T00:00:00,2016-10-04T15:13:00,2016-10-04T15:13:00,NaN,Partner,0,Intact Male,-24
45764,0 years,Cat,Domestic Shorthair Mix,White/Orange,2016-11-15T00:00:00,2016-11-05T18:16:00,2016-11-05T18:16:00,Jellybean,NaN,1,Intact Female,-10
50533,0 years,Cat,Domestic Shorthair Mix,Tortie,2015-07-06T00:00:00,2015-07-05T14:46:00,2015-07-05T14:46:00,NaN,Partner,0,Unknown,-1
16754,1 day,Cat,Domestic Shorthair Mix,Blue Cream,2014-03-31T00:00:00,2014-04-01T18:47:00,2014-04-01T18:47:00,NaN,Partner,0,Intact Female,1
32388,1 day,Cat,Domestic Shorthair Mix,Brown Tabby/White,2016-04-04T00:00:00,2016-04-05T14:42:00,2016-04-05T14:42:00,NaN,Partner,0,Intact Female,1
63655,0 years,Cat,Domestic Shorthair Mix,Orange Tabby,2016-06-28T00:00:00,2016-06-28T11:33:00,2016-06-28T11:33:00,NaN,Partner,0,Unknown,1
9835,0 years,Dog,Beagle/Chihuahua Shorthair,White,2013-11-04T00:00:00,2013-11-04T15:00:00,2013-11-04T15:00:00,NaN,Partner,0,Intact Female,1


In [19]:
data['animal_type'].unique()

array(['Cat', 'Dog', 'Other', 'Bird', 'Livestock'], dtype=object)

In [3]:
data['outcome_type'].unique()

array(['Transfer', 'Adoption', 'Euthanasia', 'Return to Owner', 'Died',
       'Disposal', 'Relocate', 'Missing', nan, 'Rto-Adopt'], dtype=object)

In [4]:
data.shape

(78256, 12)

In [5]:
import pandas_profiling

In [6]:
data.profile_report()

/Users/max/anaconda3/lib/python3.7/site-packages/pandas_profiling/model/correlations.py:124: UserWarning: There was an attempt to calculate the cramers correlation, but this failed.
To hide this warning, disable the calculation
(using `df.profile_report(correlations={"cramers": False}`)
If this is problematic for your use case, please report this as an issue:
https://github.com/pandas-profiling/pandas-profiling/issues
(include the error message: 'The internally computed table of expected frequencies has a zero element at (0, 2).')
  correlation_name=correlation_name, error=error
